# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
from api_key_store import g_key

# CSV Path
csvpath = "output_data/cities.csv"

# Places URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
wthrdf = pd.read_csv(csvpath)
wthrdf.head()

,Cities,Latitude,Longitude,Temperature (F),Pressure,Humidity (%),Minimum Temperature (F),Maximum Temperature (F),Cloud Coverage (%),Wind Speed (mph)
0,nome,30.04,-94.42,77.40,1012.0,88,75.0,80.60,90,8.05
1,hilo,19.71,-155.08,77.56,1014.0,58,66.0,86.00,75,11.41
2,kaeo,-35.10,173.78,65.35,1027.0,84,64.0,66.99,95,2.82
3,cabo san lucas,22.89,-109.91,87.75,1011.0,55,84.0,91.00,5,13.87
4,ushuaia,-54.81,-68.31,39.20,995.0,80,39.2,39.20,40,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
locations = wthrdf[["Latitude","Longitude"]]
weights = wthrdf["Humidity (%)"]        

# Plot
fig = gmaps.figure(center=(51.5,0), zoom_level=1.25)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=200, point_radius=3))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Criteria for hotels

# Less than 90F and greater than 80F, less than 30% cloud cover, spa vibe
citlst = wthrdf.loc[(wthrdf["Maximum Temperature (F)"] < 90) & (wthrdf["Minimum Temperature (F)"] > 80) & (wthrdf["Cloud Coverage (%)"] < 30)]

# Set API params and result arrays
params = {
    "type": "lodging",
    "radius": 5000,
    "keyword": "spa",
    "key": g_key,
}

hotnam = []
hotlat = []
hotlng = []
hotcit = []

for index, row in citlst.iterrows():

    # Get city locations from df
    citloc = str(row['Latitude']) + "," + str(row['Longitude'])
    params['location'] = citloc
    
    # Get hotel names and locations
    res = requests.get(base_url, params=params).json()
    if res['status'] != "ZERO_RESULTS":
        hotnam.append(res['results'][0]['name'])  
        hotlat.append(res['results'][0]['geometry']['location']['lat'])
        hotlng.append(res['results'][0]['geometry']['location']['lng'])
        hotcit.append(res['results'][0]['vicinity'])
        

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# Hotel Dataframe (pointless)
hotel_df = pd.DataFrame(hotnam)
hotel_df = hotel_df.rename(columns={ 0 : "Hotel Name"})
hotel_df["Hotel Latitude"] = pd.DataFrame(hotlat)
hotel_df["Hotel Longitude"] = pd.DataFrame(hotlng)
hotel_df["Location"] = pd.DataFrame(hotcit)

# Get city locations from df
hotloc = hotel_df[['Hotel Latitude','Hotel Longitude']]


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Location</dt><dd>{Location}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Latitude","Hotel Longitude"]]

In [33]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(hotloc,info_box_content=hotel_info))
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))